In [53]:
from IPython.display import HTML
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import precision_score
    
%matplotlib inline
import seaborn as sns

In [54]:
#read in the data
data = pd.read_csv('AggredgatedData.csv', sep=',', na_values=[" ", ""], index_col=0)

In [55]:
features = list(data.columns[1:-1])
X = data[features]
Y = data['2016ODabovenatavg']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=1)

##I. SVM

Pros: 
* Effective in high dimensional spaces.
* Still effective in cases where number of dimensions is greater than the number of samples.
* Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
* Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

Cons:

* If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.
* SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation

In [56]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [57]:
kernels = ['linear', 'polynomial', 'rbf', 'sigmoid']

for i in kernels:
    for j in range(1, 9, 2):
        clf = SVC()
        clf.fit(X_train, Y_train) 
        #kernel? C?
        SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
            decision_function_shape='ovr', degree=j, gamma='auto', kernel=i,
            max_iter=-1, probability=False, random_state=None, shrinking=True,
            tol=0.001, verbose=False)
        scores = cross_val_score(clf, X, Y, cv = 10)

        print ("Kernel: %s | Degree: %i" % (i, j))
        print scores
        print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Kernel: linear | Degree: 1
[ 0.64705882  0.75        0.6875      0.625       0.75        0.6875
  0.8125      0.625       0.6         0.73333333]
Accuracy: 0.69 (+/- 0.13)
Kernel: linear | Degree: 3
[ 0.64705882  0.75        0.6875      0.625       0.75        0.6875
  0.8125      0.625       0.6         0.73333333]
Accuracy: 0.69 (+/- 0.13)
Kernel: linear | Degree: 5
[ 0.64705882  0.75        0.6875      0.625       0.75        0.6875
  0.8125      0.625       0.6         0.73333333]
Accuracy: 0.69 (+/- 0.13)
Kernel: linear | Degree: 7
[ 0.64705882  0.75        0.6875      0.625       0.75        0.6875
  0.8125      0.625       0.6         0.73333333]
Accuracy: 0.69 (+/- 0.13)
Kernel: polynomial | Degree: 1
[ 0.64705882  0.75        0.6875      0.625       0.75        0.6875
  0.8125      0.625       0.6         0.73333333]
Accuracy: 0.69 (+/- 0.13)
Kernel: polynomial | Degree: 3
[ 0.64705882  0.75        0.6875      0.625       0.75        0.6875
  0.8125      0.625       0.6       

##II. Logistic Regression

In the multiclass case, the training algorithm uses the one-vs-rest (OvR) scheme if the ‘multi_class’ option is set to ‘ovr’, and uses the cross- entropy loss if the ‘multi_class’ option is set to ‘multinomial’. (Currently the ‘multinomial’ option is supported only by the ‘lbfgs’, ‘sag’ and ‘newton-cg’ solvers.)
This class implements regularized logistic regression using the ‘liblinear’ library, ‘newton-cg’, ‘sag’ and ‘lbfgs’ solvers. It can handle both dense and sparse input. Use C-ordered arrays or CSR matrices containing 64-bit floats for optimal performance; any other input format will be converted (and copied).
The ‘newton-cg’, ‘sag’, and ‘lbfgs’ solvers support only L2 regularization with primal formulation. The ‘liblinear’ solver supports both L1 and L2 regularization, with a dual formulation only for the L2 penalty.

In [15]:
from sklearn.linear_model import LogisticRegression

In [58]:
clf1 = LogisticRegression()
clf1.fit(X_train, Y_train) 
#kernel? C?
LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight=None, random_state=None, solver='liblinear', 
                   max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)

print(clf1.predict(X_test))

[False False False  True False  True  True False False  True  True False
 False False False False  True  True False False  True  True False False
 False False  True  True False False False False  True False False False
  True False  True False False False False False  True False  True  True
 False False  True False False]


In [59]:
scores = cross_val_score(clf1, X, Y, cv = 10)
print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.70588235  0.75        0.375       0.6875      0.875       0.625       0.8125
  0.6875      0.66666667  0.66666667]
Accuracy: 0.69 (+/- 0.25)


In [60]:
#Comparison of the sparsity (% of zero coefficients) of solutions when L1 and L2 penalty 
#are used for different values of C. 
#We can see that large values of C give more freedom to the model. 
#Conversely, smaller values of C constrain the model more. 
#In the L1 penalty case, this leads to sparser solutions.

for i, C in enumerate((100, 1, 0.01)):
    # turn down tolerance for short training time
    clf_l1_LR = LogisticRegression(C=C, penalty='l1', tol=0.01)
    clf_l2_LR = LogisticRegression(C=C, penalty='l2', tol=0.01)
    clf_l1_LR.fit(X, Y)
    clf_l2_LR.fit(X, Y)

    coef_l1_LR = clf_l1_LR.coef_.ravel()
    coef_l2_LR = clf_l2_LR.coef_.ravel()

    # coef_l1_LR contains zeros due to the
    # L1 sparsity inducing norm

    sparsity_l1_LR = np.mean(coef_l1_LR == 0) * 100
    sparsity_l2_LR = np.mean(coef_l2_LR == 0) * 100

    print("C=%.2f" % C)
    print("Sparsity with L1 penalty: %.2f%%" % sparsity_l1_LR)
    print("score with L1 penalty: %.4f" % clf_l1_LR.score(X, Y))
    print("Sparsity with L2 penalty: %.2f%%" % sparsity_l2_LR)
    print("score with L2 penalty: %.4f" % clf_l2_LR.score(X, Y))

C=100.00
Sparsity with L1 penalty: 6.67%
score with L1 penalty: 0.8616
Sparsity with L2 penalty: 0.00%
score with L2 penalty: 0.8616
C=1.00
Sparsity with L1 penalty: 50.00%
score with L1 penalty: 0.7547
Sparsity with L2 penalty: 0.00%
score with L2 penalty: 0.7862
C=0.01
Sparsity with L1 penalty: 93.33%
score with L1 penalty: 0.6352
Sparsity with L2 penalty: 0.00%
score with L2 penalty: 0.7044


##III. K-Means (2 centroids)

In [22]:
from sklearn.cluster import KMeans

In [43]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_train)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

In [41]:
kmeans.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [42]:
kmeans.cluster_centers_

array([[  6.85714286e-01,   7.14285714e-01,   9.90476190e-01,
          1.06666667e+00,   2.66666667e-01,   1.26666667e+00,
          1.45568932e-01,   1.73476153e-01,   4.51789978e-01,
         -9.63652793e-02,  -4.64647099e-01,   2.21660688e-01,
         -7.06148276e-02,   2.77903902e-02,   3.98652509e-01,
          2.35427698e-02,  -5.50511346e-03,   8.18997065e-02,
         -2.02946285e-02,  -6.40498423e-02,  -1.85447160e-01,
          6.23701023e-02,   8.74484442e-02,   9.35764354e-02,
          1.01390090e-02,   1.15824059e-02,   8.62609524e-01,
          1.19457645e-04],
       [  1.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         -1.14942529e-02,  -3.48484848e-01,   4.82758621e-01,
         -2.33333333e-01,  -3.23529412e-01,  -6.61764706e-01,
          1.65009940e-01,   1.53543307e-01,   5.85000000e+02,
          1.33924653e-03,  -1.38186319e-01,

In [45]:
##SCORING?

## IV. Stochastic Gradient Descent (SGD)

Pros:
* Efficiency.
* Ease of implementation (lots of opportunities for code tuning).

Cons:
* requires a number of hyperparameters such as the regularization parameter and the number of iterations.
* sensitive to feature scaling.

In [33]:
from sklearn.linear_model import SGDClassifier

In [61]:
clf3 = SGDClassifier(loss="hinge", penalty="l2")

clf3.fit(X_train, Y_train)
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [62]:
clf3.predict(X_test)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False], dtype=bool)

In [63]:
clf3.coef_

array([[ -5.23217789e+01,  -3.27011118e+01,   7.19424460e+01,
         -5.23217789e+01,   7.19424460e+01,   3.27011118e+01,
          3.39323448e+02,   3.81390625e+02,   2.80224415e+02,
          4.88173195e+00,   5.42808692e+00,  -1.96832908e+01,
         -2.75646968e+01,  -2.38539525e+01,  -4.16709189e+01,
          1.13993476e+02,  -3.43336912e+01,  -1.27895818e+01,
         -4.99360807e-01,   3.83814338e+01,  -9.61870462e+00,
          7.76579272e+00,   8.87467740e+00,   1.10557228e+01,
         -1.30150425e+02,  -4.07913669e+01,  -3.47670904e-01,
         -4.41077476e-01,   7.75474166e+01,   9.56902786e-02]])

In [64]:
clf3.intercept_   

array([-38.75832505])

In [65]:
scores = cross_val_score(clf3, X, Y, cv = 10)
print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.64705882  0.375       0.625       0.625       0.5         0.5         0.6875
  0.375       0.33333333  0.73333333]
Accuracy: 0.54 (+/- 0.27)
